# Trabajo final algoritmos de optimización

**Problema seleccionado: Maximización del número de partidos de la liga**

## Índice de contenidos

### [1. Introducción de puntuaciones](#section-1)
### [2. Imports](#section-2)
### [3. Implantación algoritmo voraz](#section-3)
### [4. Implantación algoritmo Grasp](#section-4)
### [5. Implantación de algoritmo genético](#section-5)

<a id="section-1"></a>
# 1. Introducción de puntuaciones

In [1]:
slots= {
    'V20': .4,
    
    'S12': .55,
    'S16': .7,
    'S18': .8,
    'S20': 1,

    'D12': .45,
    'D16': .75,
    'D18': .85,
    'D20': 1,

    'L20': .4  
}

n_coincidencias= {
                    0: 0,
                    1: .25,
                    2: .45,
                    3: .6,
                    4: .7,
                    5: .75,
                    6: .78,
                    7: .8,
                    8: .8,
                    9: .85,
                    10: .85
}
n_equipos= {'CatA': 3, 'CatB': 11, 'CatC': 6}

puntuaciones= {'CatA': {'CatA': 2, 'CatB': 1.3, 'CatC': 1},
              'CatB': {'CatA': 1.3, 'CatB': .9, 'CatC': .75},
              'CatC': {'CatA': 1, 'CatB': .75, 'CatC': .47},
              }


<a id='section-2'></a>
## 2. Imports

In [2]:
import pandas as pd
import random
import time
import numpy as np

<a id='section-3'></a>
## 3. Implantación algoritmo voraz

**Función para generar soluciones aleatorias así como función para obtener la puntuación asociada a determinada solución:**

In [3]:
def genera_aleatoria(slots, n_equipos):
    """
    Genera un horario aleatorio de partidos con sus puntuaciones asociadas.

    Args:
        slots (dict): Diccionario que contiene los horarios disponibles y sus pesos.
        n_coincidencias (dict): Diccionario que contiene el número de coincidencias y sus pesos correspondientes.
        n_equipos (dict): Diccionario que especifica el número de equipos en cada categoría.
        puntuaciones (dict): Diccionario que contiene las puntuaciones para las diferentes categorías de equipos.

    Returns:
        df (DataFrame): DataFrame que contiene los partidos generados, los horarios, las puntuaciones base y las correcciones.
        puntuacion (float): Puntuación total calculada a partir de los partidos generados.
    """

    n_equipos_c= n_equipos.copy()
    sol= []
    num_partidos= sum(list(n_equipos_c.values()))
    # loop para rellenar los partidos y los horarios
    i= 0
    while sum(list(n_equipos_c.values())) != 0:
        equipo_1_i= random.randint(0, len(n_equipos_c) - 1)
        equipo_1_name_i= list(n_equipos_c.keys())[equipo_1_i]
        n_equipos_c[list(n_equipos_c.keys())[equipo_1_i]]-= 1
        if n_equipos_c[list(n_equipos_c.keys())[equipo_1_i]]== 0:
            n_equipos_c.pop(list(n_equipos_c.keys())[equipo_1_i])
        
        equipo_2_i= random.randint(0, len(n_equipos_c) - 1)
        equipo_2_name_i= list(n_equipos_c.keys())[equipo_2_i]

        n_equipos_c[list(n_equipos_c.keys())[equipo_2_i]]-= 1
        if n_equipos_c[list(n_equipos_c.keys())[equipo_2_i]]== 0:
            n_equipos_c.pop(list(n_equipos_c.keys())[equipo_2_i])
        # generamos un horario aleatorio:
        horario_i= list(slots.keys())[random.randint(0, len(slots) - 1)]
        sol.append([equipo_1_name_i, equipo_2_name_i, horario_i])

        i+= 1
    # Rellenamos el dataframe al salir del loop:
    df= pd.DataFrame(data= sol, columns= ['equipoA', 'equipoB', 'horario_partido'])

    #nos aseguramos que haya partido tanto el lunes como el viernes:
    if 'L20' not in df['horario_partido'].values and 'V20' not in df['horario_partido'].values:
        pos_L= random.randint(0, len(df) - 1)
        df.loc[pos_L, 'horario_partido']= 'L20'
        pos_V= random.choice([i for i in range(0, len(df) - 1) if i != pos_L])
        df.loc[pos_V, 'horario_partido']= 'V20'
    elif 'L20' not in df['horario_partido'].values:
        pos_L= random.randint(0, len(df) - 1)
        df.loc[pos_L, 'horario_partido']= 'L20'
    elif 'V20' not in df['horario_partido'].values:
        pos_V= random.randint(0, len(df) - 1)
        df.loc[pos_V, 'horario_partido']= 'V20'

    return df

def obtener_puntacion(df, n_coincidencias, puntuaciones, slots):
    df.dropna(inplace= True)
    df['base']= df[['equipoA', 'equipoB']].apply(lambda x: puntuaciones[x['equipoA']][x['equipoB']], axis= 1)
    df['correccion_coincidencia']= df['horario_partido'].apply(lambda x: 1 - n_coincidencias[df['horario_partido'].value_counts().loc[x] - 1])
    df['correccion_fecha']= df['horario_partido'].apply(lambda x: slots[x])
        
    df['puntuacion']= df['base'] * df['correccion_coincidencia'] * df['correccion_fecha'].round(2)
    return df, df['puntuacion'].sum()


sol= genera_aleatoria(slots, n_equipos)
display(sol)
sol, puntuacion= obtener_puntacion(sol, n_coincidencias, puntuaciones, slots)
display(sol)
print('\n')
print(puntuacion)

,equipoA,equipoB,horario_partido
0,CatC,CatC,D12
1,CatB,CatA,S20
2,CatB,CatA,L20
3,CatA,CatB,D12
4,CatC,CatC,V20
5,CatC,CatC,S18
6,CatB,CatB,D12
7,CatB,CatB,D16
8,CatB,CatB,S16
9,CatB,CatB,S18


,equipoA,equipoB,horario_partido,base,correccion_coincidencia,correccion_fecha,puntuacion
0,CatC,CatC,D12,0.47,0.55,0.45,0.116325
1,CatB,CatA,S20,1.30,1.00,1.00,1.300000
2,CatB,CatA,L20,1.30,1.00,0.40,0.520000
3,CatA,CatB,D12,1.30,0.55,0.45,0.321750
4,CatC,CatC,V20,0.47,1.00,0.40,0.188000
5,CatC,CatC,S18,0.47,0.75,0.80,0.282000
6,CatB,CatB,D12,0.90,0.55,0.45,0.222750
7,CatB,CatB,D16,0.90,1.00,0.75,0.675000
8,CatB,CatB,S16,0.90,1.00,0.70,0.630000
9,CatB,CatB,S18,0.90,0.75,0.80,0.540000




4.795825


In [4]:
def greedy_local_search(df, column):
    df.dropna(inplace= True)
    vals= list(df[column].values)
    mejor_puntuacion= 0
    optima= df
    df_i= df

    for i in range(1, len(df) - 1):
        for j in range(i + 1, len(df)):
            vecina= vals[:i] + [vals[j]] + vals[i+1:j] + [vals[i]] + vals[j+1:]
            df_i[column]= vecina
            sol, puntuacion= obtener_puntacion(df_i, n_coincidencias, puntuaciones, slots)
            if puntuacion > mejor_puntuacion:
                    mejor_puntuacion= puntuacion
                    optima= sol
    return optima, mejor_puntuacion

**Busqueda local con multiarranque mediante un algoritmo greedy:**

In [5]:
t_ini= time.time()
optimo, mejor_puntuacion= 0, 0
while time.time() - t_ini <  3 * 60:
    sol= genera_aleatoria(slots, n_equipos)
    optimo_local_iter_eqA, puntuacion= greedy_local_search(sol, 'equipoA')
    
    optimo_local_iter_eqB, puntuacion= greedy_local_search(optimo_local_iter_eqA, 'equipoB')
    
    optimo_local_iter_horario, puntuacion= greedy_local_search(optimo_local_iter_eqB, 'horario_partido')
    
    if puntuacion > mejor_puntuacion:
        mejor_puntuacion= puntuacion
        optimo= optimo_local_iter_horario
        print('\n')
        print(f'##### Actualización de la solución, nueva mejor puntuacion {round(mejor_puntuacion, 2)} mill audiencia \n tiempo de cálculo {round(time.time() - t_ini, 2)}s')
print("Mejor puntuación y resultados óptimos tras 3min: ")
mejor_puntuacion, optimo



##### Actualización de la solución, nueva mejor puntuacion 5.27 mill audiencia 
 tiempo de cálculo 0.49s


##### Actualización de la solución, nueva mejor puntuacion 5.66 mill audiencia 
 tiempo de cálculo 0.95s


##### Actualización de la solución, nueva mejor puntuacion 6.06 mill audiencia 
 tiempo de cálculo 1.4s


##### Actualización de la solución, nueva mejor puntuacion 6.16 mill audiencia 
 tiempo de cálculo 8.64s


##### Actualización de la solución, nueva mejor puntuacion 6.53 mill audiencia 
 tiempo de cálculo 11.82s


##### Actualización de la solución, nueva mejor puntuacion 6.68 mill audiencia 
 tiempo de cálculo 43.86s


##### Actualización de la solución, nueva mejor puntuacion 6.72 mill audiencia 
 tiempo de cálculo 60.75s


##### Actualización de la solución, nueva mejor puntuacion 6.76 mill audiencia 
 tiempo de cálculo 64.5s
Mejor puntuación y resultados óptimos tras 3min: 


(6.765,
   equipoA equipoB horario_partido  base  correccion_coincidencia   
 0    CatC    CatA             L20  1.00                     1.00  \
 1    CatC    CatB             S16  0.75                     1.00   
 2    CatA    CatA             S20  2.00                     0.75   
 3    CatB    CatB             D16  0.90                     1.00   
 4    CatB    CatC             D18  0.75                     1.00   
 5    CatB    CatB             S18  0.90                     0.75   
 6    CatB    CatC             V20  0.75                     1.00   
 7    CatB    CatB             S20  0.90                     0.75   
 8    CatB    CatC             D20  0.75                     1.00   
 9    CatC    CatB             S18  0.75                     0.75   
 
    correccion_fecha  puntuacion  
 0              0.40      0.4000  
 1              0.70      0.5250  
 2              1.00      1.5000  
 3              0.75      0.6750  
 4              0.85      0.6375  
 5              0.80 

**Mejor puntuación en 6,76 millones:**

In [6]:
print('mejor puntuación ', mejor_puntuacion)
display(optimo)

mejor puntuación  6.765


,equipoA,equipoB,horario_partido,base,correccion_coincidencia,correccion_fecha,puntuacion
0,CatC,CatA,L20,1.00,1.00,0.40,0.4000
1,CatC,CatB,S16,0.75,1.00,0.70,0.5250
2,CatA,CatA,S20,2.00,0.75,1.00,1.5000
3,CatB,CatB,D16,0.90,1.00,0.75,0.6750
4,CatB,CatC,D18,0.75,1.00,0.85,0.6375
5,CatB,CatB,S18,0.90,0.75,0.80,0.5400
6,CatB,CatC,V20,0.75,1.00,0.40,0.3000
7,CatB,CatB,S20,0.90,0.75,1.00,0.6750
8,CatB,CatC,D20,0.75,1.00,1.00,0.7500
9,CatC,CatB,S18,0.75,0.75,0.80,0.4500


<a id='section-4'></a>
## 4. Implantación algoritmo Grasp

**Modificamos la búsqueda local para implementar un grasp** 

In [9]:
def grasp_local_search(df, column, alpha):
    df.dropna(inplace= True)
    vals= list(df[column].values)
    mejor_puntuacion= 0
    optima= df
    df_i= df
    sol_list= []
    df_list= []
    n= 0

    for i in range(1, len(df) - 1):
        for j in range(i + 1, len(df)):
            vecina= vals[:i] + [vals[j]] + vals[i+1:j] + [vals[i]] + vals[j+1:]
            df_i[column]= vecina
            sol, puntuacion= obtener_puntacion(df_i, n_coincidencias, puntuaciones, slots)
            sol_list.append(puntuacion)
            df_list.append(df_i)
            n+= 1
    
    # random choose value:
    n_val= random.randint(0, int(alpha * n))
    
    n_best_index= np.argsort(sol_list)[::-1][:int(alpha * n + 1)]
    df_list_n_best= [df_list[i] for i in n_best_index]
    
    sol_list= np.sort(sol_list)[::-1][:int(alpha * n + 1)]
    sol_list= sol_list[:int(alpha * n + 1)]

    optima, mejor_puntuacion= df_list_n_best[n_val], sol_list[n_val]
    return optima, mejor_puntuacion

**Procedemos a obtener el parámetro alpha óptimo para el grasp:**

In [8]:
sol_dict= {}
for alpha in np.arange(0, 1, .1):
    t_ini= time.time()
    optimo, mejor_puntuacion= 0, 0
    while time.time() - t_ini <   10:
        sol= genera_aleatoria(slots, n_equipos)
        optimo_local_iter_eqA, puntuacion= grasp_local_search(sol, 'equipoA', alpha)
        
        optimo_local_iter_eqB, puntuacion= grasp_local_search(optimo_local_iter_eqA, 'equipoB', alpha)
        
        optimo_local_iter_horario, puntuacion= grasp_local_search(optimo_local_iter_eqB, 'horario_partido', alpha)
        
        if puntuacion > mejor_puntuacion:
            mejor_puntuacion= puntuacion
            optimo= optimo_local_iter_horario
            print('\n')
            print(f'##### Actualización de la solución, nueva mejor puntuacion {round(mejor_puntuacion, 2)} mill audiencia \n tiempo de cálculo {round(time.time() - t_ini, 2)}s')
    print(f"Mejor puntuación para alpha = {alpha}:")
    print(mejor_puntuacion)
    sol_dict[alpha]= mejor_puntuacion, optimo




##### Actualización de la solución, nueva mejor puntuacion 4.91 mill audiencia 
 tiempo de cálculo 0.47s


##### Actualización de la solución, nueva mejor puntuacion 5.28 mill audiencia 
 tiempo de cálculo 1.41s


##### Actualización de la solución, nueva mejor puntuacion 5.87 mill audiencia 
 tiempo de cálculo 1.88s


##### Actualización de la solución, nueva mejor puntuacion 6.39 mill audiencia 
 tiempo de cálculo 2.36s


##### Actualización de la solución, nueva mejor puntuacion 6.51 mill audiencia 
 tiempo de cálculo 10.28s
Mejor puntuación para alpha = 0.0:
6.5142500000000005


##### Actualización de la solución, nueva mejor puntuacion 5.7 mill audiencia 
 tiempo de cálculo 0.46s


##### Actualización de la solución, nueva mejor puntuacion 6.06 mill audiencia 
 tiempo de cálculo 1.83s


##### Actualización de la solución, nueva mejor puntuacion 6.15 mill audiencia 
 tiempo de cálculo 3.26s


##### Actualización de la solución, nueva mejor puntuacion 6.82 mill audiencia 
 tiempo 

In [12]:
puntuations= [val[0] for val in sol_dict.values()]
list(sol_dict.keys())[puntuations.index(max(puntuations))]

0.1

**Procedemos a aplicar el algoritmo con el alpha óptimo de 0.1**

In [22]:
t_ini= time.time()
optimo, mejor_puntuacion= 0, 0
while time.time() - t_ini <  10 * 60:
    sol= genera_aleatoria(slots, n_equipos)
    optimo_local_iter_eqA, puntuacion= grasp_local_search(sol, 'equipoA', .1)
        
    optimo_local_iter_eqB, puntuacion= grasp_local_search(optimo_local_iter_eqA, 'equipoB', .1)
        
    optimo_local_iter_horario, puntuacion= grasp_local_search(optimo_local_iter_eqB, 'horario_partido', .1)
        
    if puntuacion > mejor_puntuacion:
        mejor_puntuacion= puntuacion
        optimo= optimo_local_iter_horario
        print('\n')
        print(f'##### Actualización de la solución, nueva mejor puntuacion {round(mejor_puntuacion, 2)} mill audiencia \n tiempo de cálculo {round(time.time() - t_ini, 2)}s')




##### Actualización de la solución, nueva mejor puntuacion 4.63 mill audiencia 
 tiempo de cálculo 0.45s


##### Actualización de la solución, nueva mejor puntuacion 4.77 mill audiencia 
 tiempo de cálculo 0.87s


##### Actualización de la solución, nueva mejor puntuacion 5.26 mill audiencia 
 tiempo de cálculo 1.32s


##### Actualización de la solución, nueva mejor puntuacion 5.85 mill audiencia 
 tiempo de cálculo 3.5s


##### Actualización de la solución, nueva mejor puntuacion 6.03 mill audiencia 
 tiempo de cálculo 7.46s


##### Actualización de la solución, nueva mejor puntuacion 6.07 mill audiencia 
 tiempo de cálculo 9.66s


##### Actualización de la solución, nueva mejor puntuacion 6.11 mill audiencia 
 tiempo de cálculo 21.18s


##### Actualización de la solución, nueva mejor puntuacion 6.53 mill audiencia 
 tiempo de cálculo 30.79s


##### Actualización de la solución, nueva mejor puntuacion 6.96 mill audiencia 
 tiempo de cálculo 61.18s


##### Actualización de la solució

Solución óptima al problema:

In [23]:
print(display(optimo))
print('\n\n', mejor_puntuacion)


,equipoA,equipoB,horario_partido,base,correccion_coincidencia,correccion_fecha,puntuacion
0,CatB,CatC,L20,0.75,1,0.40,0.3000
1,CatB,CatC,D12,0.75,1,0.45,0.3375
2,CatB,CatC,D20,0.75,1,1.00,0.7500
3,CatB,CatB,D16,0.90,1,0.75,0.6750
4,CatC,CatC,V20,0.47,1,0.40,0.1880
5,CatB,CatB,S12,0.90,1,0.55,0.4950
6,CatB,CatA,S16,1.30,1,0.70,0.9100
7,CatC,CatB,D18,0.75,1,0.85,0.6375
8,CatB,CatB,S18,0.90,1,0.80,0.7200
9,CatA,CatA,S20,2.00,1,1.00,2.0000


None


 7.013
